<a href="https://colab.research.google.com/github/hukim1112/MLDL/blob/master/object_detection/SSD_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV2
import tensorflow.keras.layers as layers
import tensorflow as tf
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt

# A. Backbone 신경망의 전이학습

In [ ]:
mobilenet_v2 = MobileNetV2(input_shape=(224,224,3), weights="imagenet", include_top=False)
tf.keras.utils.plot_model(mobilenet_v2, show_shapes=True, expand_nested=True)

# functional API를 사용해 backbone을 생성
start = mobilenet_v2.get_layer("input_1").input
end = mobilenet_v2.get_layer("block_5_add").output
backbone = tf.keras.Model(start, end)

# B. Neck 신경망 구현

In [ ]:
#input shape (BN, H, W, C)

# Sequential API를 사용해 4개의 동일한 CNN 블록을 만드시오.
# 각 블록의 변수 명은 block1, block2, block3, block4이다.

block1 = tf.keras.Sequential([tf.keras.layers.Conv2D(256, 1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU(),

                     tf.keras.layers.DepthwiseConv2D(3, strides=(2,2), padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU(),

                     tf.keras.layers.Conv2D(256, 1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU()])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,H/2,W/2,512]

block2 = tf.keras.Sequential([tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.DepthwiseConv2D(3, strides=(2,2), padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,H/4,W/4,512]
        
block3 = tf.keras.Sequential([tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.DepthwiseConv2D(3, strides=(2,2)),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU()])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,H/8,W/8,512]
        
block4 = tf.keras.Sequential([tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.DepthwiseConv2D(3, strides=(2,2)),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU()])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,H/16,W/16,512]

# C. Neck 신경망 연결

In [ ]:
input_layer = tf.keras.Input(shape=(224,224,3))
x = backbone(input_layer)
y1 = block1(x)
y2 = block2(y1)
y3 = block3(y2)
y4 = block4(y3)

print("첫 번째 feature map (None, 28, 28, 32) : ", x.shape,\
      "두 번째 feature map (None, 14, 14, 256) : ", y1.shape,\
      "세 번째 feature map (None, 7, 7, 256) : ", y2.shape,\
      "네 번쨰 feature map (None, 3, 3, 256) : ",y3.shape,\
      "다섯번째 feature map (None, 1, 1, 256) : ",y4.shape)

# D. head(classification, regression) 신경망 연결



In [ ]:
'''
    각 스케일의 특징맵에 대해 classification head와 regression head를 연결한다.
    스케일에 따라 앵커박스의 종류가 다르게 사용할 수 있고, 그것이 head 네트워크의 필터 수를 결정한다.
    예를 들어 28x28, 14x14, 7x7, 3x3, 1x1의 특징맵에 대해 각각 4, 6, 6, 6, 4 종류의 앵커 박스들을 사용한다.
    아래 코드를 해석

'''
num_classes = 3

#classification head
confs = []
conf = tf.keras.layers.Conv2D(4 * num_classes, kernel_size=3, padding='same')(x)
conf = tf.reshape(conf, [-1, 28*28*4,num_classes])
confs.append(conf)
conf = tf.keras.layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y1)
conf = tf.reshape(conf, [-1, 14*14*6,num_classes])
confs.append(conf)
conf = tf.keras.layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y2)
conf = tf.reshape(conf, [-1, 7*7*6,num_classes])
confs.append(conf)
conf = tf.keras.layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y3)
conf = tf.reshape(conf, [-1, 3*3*6,num_classes])
confs.append(conf)
conf = tf.keras.layers.Conv2D(4 * num_classes, kernel_size=1)(y4)
conf = tf.reshape(conf, [-1, 1*1*4,num_classes])
confs.append(conf)


#regression head
locs = []
loc = tf.keras.layers.Conv2D(4 * 4, kernel_size=3, padding='same')(x)
loc = tf.reshape(loc, [-1, 28*28*4,4])
print(loc.shape)
locs.append(loc)
loc = tf.keras.layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y1)
loc = tf.reshape(loc, [-1, 14*14*6,4])
print(loc.shape)
locs.append(loc)
loc = tf.keras.layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y2)
loc = tf.reshape(loc, [-1, 7*7*6,4])
print(loc.shape)
locs.append(loc)
loc = tf.keras.layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y3)
loc = tf.reshape(loc, [-1, 3*3*6,4])
print(loc.shape)
locs.append(loc)
loc = tf.keras.layers.Conv2D(4 * 4, kernel_size=1)(y4)
loc = tf.reshape(loc, [-1, 1*1*4,4])
print(loc.shape)
locs.append(loc)

confs = tf.concat(confs, axis=-2)
locs = tf.concat(locs, axis=-2)

# E. 최종 모델 빌드

In [ ]:
mobilenet_ssd = tf.keras.Model(inputs=[input_layer], outputs=[confs,locs])

In [ ]:
tf.keras.utils.plot_model(mobilenet_ssd, show_shapes=True, expand_nested=False)

In [ ]:
#여기서 빌드한 모델을 저장한 뒤 나중에 학습할 때 로드하여 쓰시오.
mobilenet_ssd.save("ssd.h5")